In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [101]:
# build the dataset

block_size = 3 
X, Y = [], []
for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
#         print(''.join(itos[i] for i in context), '--->', itos[ix]) 
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

In [102]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [8]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [11]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [31]:
C = torch.randn((27, 2))

In [14]:
C[5]

tensor([ 0.5250, -0.7214])

In [16]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([ 0.5250, -0.7214])

In [22]:
C[X]

tensor([[[-0.8145,  1.9139],
         [-0.8145,  1.9139],
         [-0.8145,  1.9139]],

        [[-0.8145,  1.9139],
         [-0.8145,  1.9139],
         [ 0.5250, -0.7214]],

        [[-0.8145,  1.9139],
         [ 0.5250, -0.7214],
         [ 1.5058,  1.0332]],

        [[ 0.5250, -0.7214],
         [ 1.5058,  1.0332],
         [ 1.5058,  1.0332]],

        [[ 1.5058,  1.0332],
         [ 1.5058,  1.0332],
         [ 0.2846,  0.5616]],

        [[-0.8145,  1.9139],
         [-0.8145,  1.9139],
         [-0.8145,  1.9139]],

        [[-0.8145,  1.9139],
         [-0.8145,  1.9139],
         [-0.2561, -0.3028]],

        [[-0.8145,  1.9139],
         [-0.2561, -0.3028],
         [ 0.6976, -0.8595]],

        [[-0.2561, -0.3028],
         [ 0.6976, -0.8595],
         [-1.8627,  0.5892]],

        [[ 0.6976, -0.8595],
         [-1.8627,  0.5892],
         [-0.6941, -0.2856]],

        [[-1.8627,  0.5892],
         [-0.6941, -0.2856],
         [-1.8627,  0.5892]],

        [[-0.6941, -0

In [24]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [25]:
W1 = torch.randn(6, 100)
b1 = torch.randn(100)

In [34]:
# torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([32, 6])

In [36]:
# torch.cat(torch.unbind(emb, 1), 1)

In [44]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [45]:
h.shape

torch.Size([32, 100])

In [46]:
W2 = torch.randn(100, 27)
b2 = torch.randn(27)

In [47]:
logits = h @ W2 + b2

In [48]:
logits.shape

torch.Size([32, 27])

In [49]:
counts = logits.exp()

In [51]:
prob = counts / counts.sum(1, keepdims=True)

In [53]:
prob[0].sum()

tensor(1.0000)

In [61]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.4884)

In [62]:
#--------at one place---------#

In [156]:
#--dividing into training, validation, testing dataset

# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
    X, Y = [], []
    for w in words:

#         #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append
    
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22740, 3]) torch.Size([22740])
torch.Size([22826, 3]) torch.Size([22826])


In [157]:
Xtr.shape, Ytr.shape

(torch.Size([182580, 3]), torch.Size([182580]))

In [158]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator = g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn((100), generator = g)
W2 = torch.randn((100, 27), generator = g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [159]:
sum(p.nelement() for p in parameters)

3481

In [160]:
for p in parameters:
    p.requires_grad = True

In [166]:
for _ in range(1000):
    #minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    #Forward Pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 6)@W1+b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])
    
    #Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #Update

    for p in parameters:
        p.data += -0.01 * p.grad
print(loss.item())

2.373981475830078


In [124]:
# loss.item()

In [169]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 6)@W1+b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

2.4746079444885254

torch.Size([182691, 3]) torch.Size([182691])
torch.Size([22793, 3]) torch.Size([22793])
torch.Size([22662, 3]) torch.Size([22662])
